# AIMS Watershed Query: AnnAGNPS files preparation
This notebook replicates the data flow of what a user would do when clicking on the AIMS web interface

##### Main library imports

In [1]:
from pathlib import Path
import json
import itertools

import pyagnps

import geopandas as gpd
import pandas as pd

from tqdm import tqdm

from sqlalchemy import URL, create_engine, text as sql_text
from sqlalchemy.orm import sessionmaker

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

## User input here

### Name and description

In [2]:
# watershed_name = 'Pelahatchie Creek'
# watershed_description = 'As generated by AIMS'
# watershed_location = ''

watershed_name = 'Upper Pearl River Basin'
watershed_description = 'As generated by AIMS'
watershed_location = 'Mississippi'

### Outlet coordinates

In [3]:
lon, lat = -89.92829, 32.52256 # Upper Pearl River upstream of reservoir
# lon, lat = -89.96793, 32.38881 # Pelahatchie Creek

### Time bounds and climate method

In [4]:
climate_method = 'nldas2' # valid options ares 'nldas2', 'cmip5'

# Time period in YYYY-MM-DD format (both bounds included)
startDate="2010-01-01" 
endDate="2022-12-31"

### Main output directory

In [5]:
# output_folder = Path('C:/Users/Luc/Desktop/pelahatchie_creek')
output_folder = Path('C:/Users/Luc/Desktop/UpperPearlRiverBasin')


### Paths to useful static files

In [6]:
# Database credentials
credentials = Path("../../inputs/db_credentials.json")

# GeoPackage that contains the centroid of each NLDAS-2 grid
path_nldas_grid_centroids = Path("D:/AIMS/Datasets/Climate/NLDAS2/NLDAS2_GRID_CENTROIDS_epsg4326.gpkg")
# path_nldas_grid_centroids = Path("C:/Users/Luc/projects/pyagnps/inputs/climate/NLDAS2_GRID_CENTROIDS_epsg4326.gpkg")

# Path to root folder containing variables of the CMIP5 MACAv2METDATA
path_to_cmip5_historical_and_rcp45 = Path("D:/AIMS/Datasets/Climate/CMIP/CMIP5/MACAv2METDATA/CNRM-CM5/r1i1p1/")
path_to_cmip5_raster_points_clim_id = path_to_cmip5_historical_and_rcp45 / "cmip5_maca_v2_metdata_pts_clim_ids.gpkg"

# Path to SCS Storm Types
path_to_scs_storm_types = Path('D:/AIMS/Datasets/Climate/TR-55/scs_storm_types.gpkg')

# Path to RUSLE2 Precipitation Zones
path_to_precip_zones = Path('D:/AIMS/Datasets/Management/RUSLE2_Climate/RUSLE2-Climate-Data-20231117/precip_zones_RUSLE2_cleaned_manually_extrapolated_pchip_linear_US_units.gpkg')


## Main process starts here

### Setting up database connection

In [7]:
# DATABASE SETUP
with open(credentials, "r") as f:
    credentials = json.load(f)

user = credentials["user"]
password = credentials["password"]
host = credentials["host"]
port = credentials["port"]
database = credentials["database"]

url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

# create a SQLAlchemy engine object
engine = create_engine(url_object)

### Define output directories

In [8]:
output_folder.mkdir(exist_ok=True)

def make_annagnps_inputs_dirs(output_folder, subdirs=['general', 'climate', 'simulation', 'watershed', 'GIS']):
    subdirs_paths = []
    for subdir in subdirs:
        category_dir = output_folder / subdir
        category_dir.mkdir(exist_ok=True)
        subdirs_paths.append(category_dir)
    return subdirs_paths
        

general_dir, \
climate_dir, \
simulation_dir, \
watershed_dir, \
gis_dir = make_annagnps_inputs_dirs(output_folder)

### Identify T-HUC containing the watershed of interest

In [9]:
thuc = pd.read_sql_query(sql_text(f"SELECT thuc_near_run_id_tr({lon},{lat})"),con=engine.connect())
thuc_id = thuc.iloc[0].values[0]

### Query reach and cell data sections along with their geometries

#### Cells geometry

In [10]:
cells_query = f"SELECT geom, cell_id FROM thuc_cell_geo_tr({lon},{lat}, '{thuc_id}')"

cells_geometry = gpd.read_postgis(sql=sql_text(cells_query), con=engine.connect(), geom_col='geom')
cells_list = cells_geometry['cell_id'].unique() # List of cell_ids


#### Reach Geometry

In [11]:
reaches_query = f"SELECT geom, reach_id FROM thuc_reach_geo_tr({lon},{lat}, '{thuc_id}')"

reaches_geometry = gpd.read_postgis(sql=sql_text(reaches_query), con=engine.connect(), geom_col='geom')
reaches_list = reaches_geometry['reach_id'].unique()

#### Cell Data Section

In [12]:
query = f"SELECT * FROM thuc_{thuc_id}_annagnps_cell_data_section WHERE cell_id in {*cells_list,}"

df_cells = pd.read_sql_query(sql=sql_text(query), con=engine.connect())

#### Reach Data Section

In [13]:
query = f"SELECT * FROM thuc_{thuc_id}_annagnps_reach_data_section WHERE reach_id in {*reaches_list,}"

df_reaches = pd.read_sql_query(sql=sql_text(query), con=engine.connect())

Make reach data section "valid" for AnnAGNPS i.e. add an "OUTLET" row at the top

In [14]:
def make_df_reaches_valid(df_reaches):
    reaches = set(df_reaches['reach_id'])
    receiving_reaches = set(df_reaches['receiving_reach'])

    outlet_reach = list(receiving_reaches - reaches)[0]
    # print(f"Outlet reach: {outlet_reach}")

    outlet_row = df_reaches[df_reaches['receiving_reach']==outlet_reach].copy()
    outlet_row['reach_id'] = outlet_reach
    outlet_row['receiving_reach'] = 'OUTLET'
    outlet_row['length'] = 0

    df_reaches_valid = pd.concat([outlet_row, df_reaches], ignore_index=True)
    return df_reaches_valid

In [15]:
df_reaches_valid = make_df_reaches_valid(df_reaches)

#### Join data sections with their respective geometries

In [16]:
cells_geometry = cells_geometry.merge(df_cells, on='cell_id')
reaches_geometry = reaches_geometry.merge(df_reaches, on='reach_id')

### Query Soil data

Query soil_data and soil_layers_daya for matching soil_id as well as raw soil data

In [17]:
soil_ids_list = df_cells['soil_id'].unique()

In [18]:
query_soil = f"""SELECT * FROM usa_valid_soil_data WHERE "Soil_ID" in {*soil_ids_list,}"""
query_soil_layers = f"""SELECT * FROM usa_valid_soil_layers_data WHERE "Soil_ID" in {*soil_ids_list,}"""
query_raw = f"""SELECT * FROM raw_nrcs_soil_data WHERE "mukey" in {*soil_ids_list,}"""

df_soil_data = pd.read_sql_query(sql=sql_text(query_soil), con=engine.connect())

df_soil_layers_data = pd.read_sql_query(sql=sql_text(query_soil_layers), con=engine.connect())
df_soil_layers_data = df_soil_layers_data.sort_values(by=['Soil_ID','Layer_Number'])

df_raw = pd.read_sql_query(sql=sql_text(query_raw), con=engine.connect())

### Query Management Data

#### Get Management Field IDs

In [19]:
mgmt_field_ids_list = df_cells['mgmt_field_id'].unique()

In [20]:
query_mgmt_field_data = f"""SELECT * FROM annagnps_mgmt_field WHERE "Field_ID" in {*mgmt_field_ids_list,}"""
df_mgmt_field = pd.read_sql_query(sql=sql_text(query_mgmt_field_data), con=engine.connect())
df_mgmt_field.head()

,Field_ID,Landuse_Type_ID,Mgmt_Schd_ID,Greg_Yr_for_1st_Yr_of_Rotation,Percent_Rock_Cover,Interrill_Erosion_Code,Random_Roughness,Terrace_Horizontal_Distance,Terrace_Grade,Tile_Drain_ID,Input_Units_Code,CDL_Category,CDL_Value,Modified_CDL_Category
0,Corn,CROPLAND,Corn,None,None,None,None,None,None,None,1,Corn,1,Corn
1,Cotton,CROPLAND,Cotton,None,None,None,None,None,None,None,1,Cotton,2,Cotton
2,Dbl_Crop_WinWht_Corn,CROPLAND,Dbl_Crop_WinWht_Corn,None,None,None,None,None,None,None,1,Dbl Crop WinWht/Corn,225,Dbl_Crop_WinWht_Corn
3,Dbl_Crop_WinWht_Soybeans,CROPLAND,Dbl_Crop_WinWht_Soybeans,None,None,None,None,None,None,None,1,Dbl Crop WinWht/Soybeans,26,Dbl_Crop_WinWht_Soybeans
4,Developed_Open_Space,URBAN,Developed_Open_Space,None,None,None,None,None,None,None,1,Developed/Open Space,121,Developed_Open_Space


#### Get matching Management Schedule IDs

In [21]:
mgmt_schedule_ids_list = df_mgmt_field['Mgmt_Schd_ID'].unique()

In [22]:
query_mgmt_field_data = f"""SELECT * FROM annagnps_mgmt_schd WHERE "Mgmt_Schd_ID" in {*mgmt_schedule_ids_list,}"""
df_mgmt_schd = pd.read_sql_query(sql=sql_text(query_mgmt_field_data), con=engine.connect())

#### Get matching Crop, Non-Crop, Management Operation, and Runoff Curve Data

In [23]:
mgmt_crop_ids_list     = df_mgmt_schd['New_Crop_ID'].dropna().unique()
mgmt_non_crop_ids_list = df_mgmt_schd['New_Non-Crop_ID'].dropna().unique()
mgmt_oper_ids_list     = df_mgmt_schd['Mgmt_Operation_ID'].dropna().unique()
roc_ids_list           = df_mgmt_schd['Curve_Number_ID'].dropna().unique()

In [24]:
query_mgmt_crop_data        = f"""SELECT * FROM annagnps_crop WHERE "Crop_ID" in {*mgmt_crop_ids_list,}"""
query_mgmt_crop_growth_data = f"""SELECT * FROM annagnps_crop_growth WHERE "Crop_Growth_ID" in {*mgmt_crop_ids_list,}"""

query_mgmt_non_crop_data    = f"""SELECT * FROM annagnps_non_crop WHERE "Non-Crop_ID" in {*mgmt_non_crop_ids_list,}"""
query_mgmt_oper_data        = f"""SELECT * FROM annagnps_mgmt_oper WHERE "Mgmt_Operation_ID" in {*mgmt_oper_ids_list,}"""

query_roc_data              = f"""SELECT * FROM annagnps_runoff_curve WHERE "Curve_Number_ID" in {*roc_ids_list,}"""

df_mgmt_crop        = pd.read_sql_query(sql=sql_text(query_mgmt_crop_data), con=engine.connect())
df_mgmt_crop_growth = pd.read_sql_query(sql=sql_text(query_mgmt_crop_growth_data), con=engine.connect())
df_mgmt_non_crop    = pd.read_sql_query(sql=sql_text(query_mgmt_non_crop_data), con=engine.connect())
df_mgmt_oper        = pd.read_sql_query(sql=sql_text(query_mgmt_oper_data), con=engine.connect())
df_roc              = pd.read_sql_query(sql=sql_text(query_roc_data), con=engine.connect())

### Watershed scale variables - Simulation Period Data

#### Water boundaries

In [25]:
bounds = cells_geometry[['geom']].copy(deep=True)
bounds.geometry = bounds.geometry.buffer(0.00001)
bounds = bounds.dissolve()
bounds = bounds.to_crs('epsg:4326')

#### Watershed SCS Storm Type

In [26]:
scs_storm_types = gpd.read_file(path_to_scs_storm_types)
scs_storm_types = scs_storm_types.to_crs('epsg:4326')

In [27]:
bounds_scs = bounds.overlay(scs_storm_types)
bounds_scs['area'] = bounds_scs.geometry.area

In [28]:
main_storm_type = bounds_scs.loc[bounds_scs['area'].argmax(), 'SCS Zone Type']

#### 10_Year_EI, EI_Zone, R_factor

In [29]:
precip_zones = gpd.read_file(path_to_precip_zones)

In [30]:
bounds_precip = bounds.overlay(precip_zones)
bounds_precip['area'] = bounds_precip.geometry.area

In [31]:
weighted_R_fctr = (bounds_precip['area'] * bounds_precip['R_factor']).sum() / bounds_precip['area'].sum()
weighted_10_year_EI = (bounds_precip['area'] * bounds_precip['10_year_EI']).sum() / bounds_precip['area'].sum()

dominant_EI = bounds_precip.loc[bounds_precip['area'].argmax(), 'EI_Zone']

if dominant_EI == 'default':
    dominant_EI = pyagnps.constants.DEFAULT_EI_NUMBER # 100 
else:
    dominant_EI = int(dominant_EI.replace('US_',''))

### Watershed Data 

In [32]:
WATERSHED_DATA = {
    'Wshd_Name': watershed_name,
    'Wshd_Description': watershed_description,
    'Wshd_Location': watershed_location,
    'Latitude': lat,
    'Longitude': lon
}

### Generate climate files

Get watershed centroid computed in terms of lat and lon

In [33]:
# /!\ This centroid is computed in degrees coordinates
lon0, lat0 = cells_geometry.dissolve().centroid.x[0], cells_geometry.dissolve().centroid.y[0] 

Reset cells secondary climate file id (just in case)

In [34]:
cells_geometry['secondary_climate_file_id'] = None

#### Using NLDAS-2

Identify the NLDAS-2 grid ID nearest to each cell and populate it in the `secondary_climate_file_id`

In [35]:
if climate_method == 'nldas2':
    nldas_centroids = gpd.read_file(path_nldas_grid_centroids)
    cells_geometry = cells_geometry.sjoin_nearest(nldas_centroids)
    cells_geometry['secondary_climate_file_id'] = cells_geometry['nldas2_grid_ID']
    cells_geometry.drop(columns=['nldas2_grid_ID', 'index_right'], inplace=True)

    secondary_climate_ids = cells_geometry.loc[:,['cell_id', 'secondary_climate_file_id']]\
                            .drop_duplicates(subset='cell_id')\
                            .set_index('cell_id')
    
    # Generate climate data for the unique NLDAS-2 grid ID featuring in the watershed 
    climate_station_points = nldas_centroids[nldas_centroids['nldas2_grid_ID'].isin(cells_geometry['secondary_climate_file_id'].unique())]
    climate_station_points = climate_station_points.to_crs('epsg:4326')

    # Update df_cells with data in secondary_climate_ids
    if df_cells.index.name != 'cell_id':
        df_cells = df_cells.set_index('cell_id')

    df_cells.update(secondary_climate_ids)
    df_cells = df_cells.reset_index()

#### Using CMIP5 data

Load CMIP5 data

In [36]:
if climate_method == 'cmip5':
    clm = pyagnps.climate.ClimateAnnAGNPSCoords(coords=None,
                                                start=startDate,
                                                end=endDate)

    clm.read_cmip5_maca_data(path_to_cmip5_historical_and_rcp45.glob("*/*.nc"))

Generate or load geopackage file containing the centroids of each raster points of the CMIP5 dataset

In [37]:
if climate_method == 'cmip5':
    if path_to_cmip5_raster_points_clim_id.exists():
        print('Reading points and clim_id of CMIP5 MACAv2METDATA')
        cmip_pts = gpd.read_file(path_to_cmip5_raster_points_clim_id)
    else:
        # Generate all the possible pairs of points
        print('Generating points and clim_id of CMIP5 MACAv2METDATA')
        lat_vals = clm.ds['lat'].values
        lon_vals = clm.ds['lon'].values - 360 # For CMIP6 and CMIP5 the longitude needs to be in the [0, 360[ range, here we bring it back in the -180, 180

        all_lon_lat_pairs = list(itertools.product(lon_vals, lat_vals))

        lon_values, lat_values = zip(*all_lon_lat_pairs)

        pts = gpd.points_from_xy(x=lon_values, y=lat_values, crs=4326)
        cmip_pts = gpd.GeoDataFrame(geometry=pts, crs=4326)

        cmip_pts['clim_id'] = cmip_pts['geometry'].apply(lambda geom: clm.generate_cmip_lon_lat_secondary_climate_id((geom.x, geom.y)))

        cmip_pts.to_file(path_to_cmip5_raster_points_clim_id, driver='GPKG')

Assigning secondary climate id for every cell based on the CMIP5 data

In [38]:
if climate_method == 'cmip5':

    cells_geometry = cells_geometry.sjoin_nearest(cmip_pts)
    cells_geometry['secondary_climate_file_id'] = cells_geometry['clim_id']
    cells_geometry.drop(columns=['clim_id', 'index_right'], inplace=True)


    secondary_climate_ids = cells_geometry.loc[:,['cell_id', 'secondary_climate_file_id']]\
                                .drop_duplicates(subset='cell_id')\
                                .set_index('cell_id')
    
    # List of points for which to extract the climate data for this watershed
    climate_station_points = cmip_pts[cmip_pts['clim_id'].isin(secondary_climate_ids['secondary_climate_file_id'])]
    climate_station_points = climate_station_points.to_crs('epsg:4326')

    # Update df_cells with data in secondary_climate_ids
    if df_cells.index.name != 'cell_id':
        df_cells = df_cells.set_index('cell_id')

    df_cells.update(secondary_climate_ids)
    df_cells = df_cells.reset_index()

#### Generate climate data dictionary for selected climate source

In [40]:
climate_data = {}

for feature in tqdm(climate_station_points.iterfeatures(), total=len(climate_station_points)):

    x, y = feature['geometry']['coordinates']

    if climate_method == 'nldas2':
        clim_id = feature['properties']['nldas2_grid_ID']
        climate_station_name = f"NLDAS-2 Grid ID {clim_id}"

        clm = pyagnps.climate.ClimateAnnAGNPSCoords(coords=(x,y), start=startDate, end=endDate, date_mode="local")
        df = clm.query_nldas2_generate_annagnps_climate_daily()
        
    elif climate_method == 'cmip5':
        clim_id = feature['properties']['clim_id']
        climate_station_name = f"CMIP5 MACAv2METDATA raster ID {clim_id}"
        
        clm.update_coords_start_end_dates(coords=(x,y), start=startDate, end=endDate, date_mode="local")
        df = clm.generate_annagnps_daily_climate_data_cmip5_maca()
        

    climate_data[clim_id] = {
        'data': df, # DataFrame containing the climate data
        'climate_station': { # Climate station metadata
            'output_filepath': climate_dir / f'climate_station_{clim_id}.csv',
            'climate_station_name': climate_station_name,
            'beginning_climate_date': clm.start.strftime("%m/%d/%Y"),
            'ending_climate_date': clm.end.strftime("%m/%d/%Y"),
            'latitude': y,
            'longitude': x,
            'elevation': cells_geometry.loc[cells_geometry['secondary_climate_file_id'] == clim_id, 'avg_elevation'].mean()
            }
    }

# A "global" climate daily / station data needs to be produced so the last climate file will be used
climate_data['global'] = climate_data[clim_id]
climate_data['global']['climate_station']['output_filepath'] = climate_dir / 'climate_station.csv'

 42%|████▏     | 27/64 [09:53<13:33, 21.98s/it]


KeyError: Timestamp('2022-12-31 06:00:00')

## Export everything to files

#### Reaches and Cells

Data sections

In [41]:
cells_path = watershed_dir / 'cell_data_section.csv'
reaches_path = watershed_dir / 'reach_data_section.csv'

df_cells.to_csv(cells_path, index=False, float_format='%1.5f')
df_reaches_valid.to_csv(reaches_path, index=False, float_format='%1.5f')

GIS layers

In [42]:
cells_geometry.to_file(gis_dir / 'cells_geometry.gpkg', driver='GPKG', index=False)
reaches_geometry.to_file(gis_dir / 'reaches_geometry.gpkg', driver='GPKG', index=False)

#### Soil Data Section

In [43]:
soil_data_path = general_dir / 'soil_data.csv'
soil_layers_data_path = general_dir / 'soil_layers_data.csv'
raw_soil_data_path = general_dir / 'raw_soil_data_gNATSGO.csv'

df_soil_data.to_csv(soil_data_path, index=False)
df_soil_layers_data.to_csv(soil_layers_data_path, index=False)
df_raw.to_csv(raw_soil_data_path, index=False)

#### Management Data

In [54]:
crop_data_path = general_dir / 'crop_data.csv'
crop_growth_path = general_dir / 'crop_growth.csv'
non_crop_path = general_dir / 'non_crop.csv'
management_field_path = general_dir / 'management_field.csv'
management_schedule_path = general_dir / 'management_schedule.csv'
management_operation_path = general_dir / 'management_oper.csv'
roc_path = general_dir / 'runoffcurve.csv'

df_mgmt_oper = pyagnps.annagnps.format_mgmt_operation_for_output(df_mgmt_oper)
df_mgmt_schd = pyagnps.annagnps.format_mgmt_schedule_for_output(df_mgmt_schd)

df_mgmt_crop.to_csv(crop_data_path, index=False)
df_mgmt_crop_growth.to_csv(crop_growth_path, index=False)
df_mgmt_non_crop.to_csv(non_crop_path, index=False)

df_mgmt_field.to_csv(management_field_path, index=False)
df_mgmt_schd.to_csv(management_schedule_path, index=False)
df_mgmt_oper.to_csv(management_operation_path, index=False)


df_roc.to_csv(roc_path, index=False)

#### Climate

In [35]:
for clim_id in climate_data:
    # Climate Daily
    if clim_id == 'global':
        climate_path = climate_dir / f"climate_daily.csv"
    else:
        climate_path = climate_dir / f"climate_daily_{clim_id}.csv"

    climate_data[clim_id]['data'].to_csv(climate_path, index=False, float_format="%.3f")
    # Climate Station
    pyagnps.climate.generate_climate_station_file(**climate_data[clim_id]['climate_station'])

#### Watershed Data

In [44]:
watershed_path = watershed_dir / 'watershed_data.csv'

pyagnps.utils.write_csv_from_dict(WATERSHED_DATA, output_path=watershed_path)

#### Global IDs Factors and Flag Data

In [45]:
DEFAULT_GLOBAL_FACTORS_FLAGS = pyagnps.constants.DEFAULT_GLOBAL_FACTORS_FLAGS
DEFAULT_GLOBAL_FACTORS_FLAGS['Wshd_Storm_Type_ID'] = main_storm_type

In [46]:
globfac_path = simulation_dir / 'globfac.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_GLOBAL_FACTORS_FLAGS, globfac_path)

#### Output Options - Global

In [47]:
DEFAULT_OUTPUT_OPTIONS_GLOBAL = pyagnps.constants.DEFAULT_OUTPUT_OPTIONS_GLOBAL

outopts_global_path = simulation_dir / 'outopts_global.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_OUTPUT_OPTIONS_GLOBAL, outopts_global_path)

#### Output Options - AA

In [48]:
DEFAULT_OUTPUT_OPTIONS_AA = pyagnps.constants.DEFAULT_OUTPUT_OPTIONS_AA

outopts_aa_path = simulation_dir / 'outopts_aa.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_OUTPUT_OPTIONS_AA, outopts_aa_path)

#### Output Options - TABLE

In [49]:
DEFAULT_OUTPUT_OPTIONS_TBL = pyagnps.constants.DEFAULT_OUTPUT_OPTIONS_TBL

outopts_tbl_path = simulation_dir / 'outopts_tbl.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_OUTPUT_OPTIONS_TBL, outopts_tbl_path)

#### AnnAGNPS ID file

In [50]:
DEFAULT_ANNAGNPS_ID = pyagnps.constants.DEFAULT_ANNAGNPS_ID

annaid_path = simulation_dir / 'annaid.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_ANNAGNPS_ID, annaid_path)

#### Simulation Period Data

In [51]:
DEFAULT_SIM_PERIOD_DATA = pyagnps.constants.DEFAULT_SIM_PERIOD_DATA

DEFAULT_SIM_PERIOD_DATA['Simulation_Begin_Year'] = clm.start.year
DEFAULT_SIM_PERIOD_DATA['Simulation_Begin_Month'] = clm.start.month
DEFAULT_SIM_PERIOD_DATA['Simulation_Begin_Day'] = clm.start.day

DEFAULT_SIM_PERIOD_DATA['Simulation_End_Year'] = clm.end.year
DEFAULT_SIM_PERIOD_DATA['Simulation_End_Month'] = clm.end.month
DEFAULT_SIM_PERIOD_DATA['Simulation_End_Day'] = clm.end.day

DEFAULT_SIM_PERIOD_DATA['Rainfall_Fctr'] = weighted_R_fctr
DEFAULT_SIM_PERIOD_DATA['10-Year_EI'] = weighted_10_year_EI
DEFAULT_SIM_PERIOD_DATA['EI_Number'] = dominant_EI

sim_period_path = simulation_dir / 'sim_period.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_SIM_PERIOD_DATA, sim_period_path)

#### AnnAGNPS Master File

In [52]:
def relative_input_file_path(output_folder, path_to_file):
    path_to_file = Path(path_to_file)
    output_folder = Path(output_folder)
    relative_path = str(path_to_file.relative_to(output_folder))
    relative_path = relative_path.replace('\\', '/')
    relative_path = f"./{relative_path}"
    return relative_path

In [55]:
master_file = {
    'AnnAGNPS ID': relative_input_file_path(output_folder, annaid_path),
    'Cell Data': relative_input_file_path(output_folder, cells_path),
    'Crop Data': relative_input_file_path(output_folder, crop_data_path),
    'Crop Growth Data': relative_input_file_path(output_folder, crop_growth_path),
    'Global IDs Factors and Flags Data': relative_input_file_path(output_folder, globfac_path),
    'Management Field Data': relative_input_file_path(output_folder, management_field_path),
    'Management Operation Data': relative_input_file_path(output_folder, management_operation_path),
    'Management Schedule Data': relative_input_file_path(output_folder, management_schedule_path),
    'Non-Crop Data': relative_input_file_path(output_folder, non_crop_path),
    'Reach Data': relative_input_file_path(output_folder, reaches_path),
    'Runoff Curve Number Data': relative_input_file_path(output_folder, roc_path),
    'Simulation Period Data': relative_input_file_path(output_folder, sim_period_path),
    'Soil Data': relative_input_file_path(output_folder, soil_data_path),
    'Soil Layer Data': relative_input_file_path(output_folder, soil_layers_data_path),
    'Watershed Data': relative_input_file_path(output_folder, watershed_path),
    'Output Options - Global': relative_input_file_path(output_folder, outopts_global_path),
    'Output Options - AA': relative_input_file_path(output_folder, outopts_aa_path),
    'Output Options - TBL': relative_input_file_path(output_folder, outopts_tbl_path),
    'CLIMATE DATA - DAILY': relative_input_file_path(output_folder, climate_dir / 'climate_daily.csv'),
    'CLIMATE DATA - STATION': relative_input_file_path(output_folder, climate_dir / 'climate_station.csv')
    # 'Output Options - Reach'
}

df_master = pd.DataFrame({
    'Data Section ID': master_file.keys(),
    'File Name': master_file.values()})

df_master.to_csv(output_folder / 'annagnps_master.csv', index=False)

### AnnAGNPS.fil file

In [56]:
annagnps_fil = output_folder / 'AnnAGNPS.fil'
annagnps_fil.write_text('annagnps_master.csv');